# RQ4 - Which DBMSs are often used together

Old code to filter sequential patterns using SPMF results

Uses:

- resources/historical_rulesv1.xlsx (from rq4_synergy/historical_coocurrence_version1.ipynb **manual**)
- resources/historical_rulesv5.xlsx (from rq4_synergy/historical_coocurrence_version5.ipynb **manual**)
- resources/historical_rulesv10.xlsx (from rq4_synergy/historical_coocurrence_version10.ipynb **manual**)
- It is necessary to modify the variable (projectsWithBD) used in the calculations of the values: antecedent_sup_abs, consequent_sup_abs and support_abs. Use the number of projects where we found indications of database usage in history.

In [1]:
import pandas as pd
import numpy as np
import os
from util import HISTORICAL_FILE_JOIN, RESOURCE_DIR 
#pip install mlxtend
from mlxtend.frequent_patterns import apriori, association_rules

 Generates association rules for the results found in the projects history first slice.

In [2]:
#df = pd.read_excel("historical_join.xlsx", keep_default_na=False)
df = pd.read_excel(HISTORICAL_FILE_JOIN, keep_default_na=False)

#len(df)
df.head()

,owner,name,domain,sha1,part_commit,date_commit,isLast,SapHana,Realm,SAP SQL Anywhere,...,Firebird,Cassandra,Couchbase,Etcd,PostGIS,Redis,Join_Ignite-Sql,Join_Ignite-NoSql,Join_Virtuoso-Sql,Join_Virtuoso-NoSql
0,Activiti,Activiti,Enterprise Resource Planning,7262dc3913045001cbc658a8474e9d295980c345,1,2010-07-14 06:36:18+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Activiti,Activiti,Enterprise Resource Planning,38b284602be9dcb6b4253cfa1c2f9420c002b56a,2,2010-08-03 08:31:19+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Activiti,Activiti,Enterprise Resource Planning,fbcdc18a66e7f7960a7ed966f87b1527c5c7551a,3,2010-09-02 09:45:24+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Activiti,Activiti,Enterprise Resource Planning,8d05dda18ce4c5db6f30cf01d7e0c2059b1a5388,4,2010-09-16 15:33:33+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Activiti,Activiti,Enterprise Resource Planning,a789ba2b30383777b4d0d81bbc62a90a24dbb032,5,2010-09-23 13:38:28+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
df2 = df.copy()
df2

,owner,name,domain,sha1,part_commit,date_commit,isLast,SapHana,Realm,SAP SQL Anywhere,...,Firebird,Cassandra,Couchbase,Etcd,PostGIS,Redis,Join_Ignite-Sql,Join_Ignite-NoSql,Join_Virtuoso-Sql,Join_Virtuoso-NoSql
0,Activiti,Activiti,Enterprise Resource Planning,7262dc3913045001cbc658a8474e9d295980c345,1,2010-07-14 06:36:18+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Activiti,Activiti,Enterprise Resource Planning,38b284602be9dcb6b4253cfa1c2f9420c002b56a,2,2010-08-03 08:31:19+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Activiti,Activiti,Enterprise Resource Planning,fbcdc18a66e7f7960a7ed966f87b1527c5c7551a,3,2010-09-02 09:45:24+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Activiti,Activiti,Enterprise Resource Planning,8d05dda18ce4c5db6f30cf01d7e0c2059b1a5388,4,2010-09-16 15:33:33+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Activiti,Activiti,Enterprise Resource Planning,a789ba2b30383777b4d0d81bbc62a90a24dbb032,5,2010-09-23 13:38:28+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31111,liferay,liferay-portal,Enterprise Resource Planning,fcd4cd6526c80613eca69927effe931bb6a91444,8209,2024-10-07 12:05:23+00:00,False,0,0,0,...,0,0,0,0,1,1,0,0,0,0
31112,liferay,liferay-portal,Enterprise Resource Planning,cc0ab8103195e543056639c8c2883fcc187c33a2,8210,2024-10-07 17:58:58+00:00,False,0,0,0,...,0,0,0,0,1,1,0,0,0,0
31113,liferay,liferay-portal,Enterprise Resource Planning,f75082029372ab28694650bd3fe9fae4d548351e,8211,2024-10-11 04:36:52+00:00,False,0,0,0,...,0,0,0,0,1,1,0,0,0,0
31114,liferay,liferay-portal,Enterprise Resource Planning,a80a679a86d2867f985bafa510d9d12f1bf051a7,8212,2024-10-11 16:53:22+00:00,False,0,0,0,...,0,0,0,0,1,1,0,0,0,0


Grouping the lines to identify which projects that don't have DB clues

In [4]:
df2 = df2.groupby(["owner", "name"]).sum()
df2


,,part_commit,isLast,SapHana,Realm,SAP SQL Anywhere,FileMaker,ArangoDB,Informix,HyperSQL,CouchDB,...,Firebird,Cassandra,Couchbase,Etcd,PostGIS,Redis,Join_Ignite-Sql,Join_Ignite-NoSql,Join_Virtuoso-Sql,Join_Virtuoso-NoSql
owner,name,,,,,,,,,,,,,,,,,,,,,
Activiti,Activiti,1540,0,0,0,0,0,0,0,13,0,...,0,0,0,0,0,0,0,0,0,0
Alluxio,alluxio,5886,0,0,0,0,0,0,0,0,0,...,0,0,0,4,0,0,0,0,0,0
AlmasB,FXGL,105,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AntennaPod,AntennaPod,703,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AnySoftKeyboard,AnySoftKeyboard,703,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
xuxueli,xxl-job,120,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
yacy,yacy_search_server,5151,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
zaproxy,zaproxy,666,0,0,0,0,0,0,0,36,0,...,0,0,0,0,0,0,0,0,0,0


Reseting the index column

In [5]:
df2 = df2.reset_index()
df2

,owner,name,part_commit,isLast,SapHana,Realm,SAP SQL Anywhere,FileMaker,ArangoDB,Informix,...,Firebird,Cassandra,Couchbase,Etcd,PostGIS,Redis,Join_Ignite-Sql,Join_Ignite-NoSql,Join_Virtuoso-Sql,Join_Virtuoso-NoSql
0,Activiti,Activiti,1540,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Alluxio,alluxio,5886,0,0,0,0,0,0,0,...,0,0,0,4,0,0,0,0,0,0
2,AlmasB,FXGL,105,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,AntennaPod,AntennaPod,703,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,AnySoftKeyboard,AnySoftKeyboard,703,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
357,xuxueli,xxl-job,120,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
358,yacy,yacy_search_server,5151,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
359,zaproxy,zaproxy,666,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
360,zendesk,maxwell,105,0,0,0,0,0,0,0,...,0,0,0,0,0,7,0,0,0,0


In [6]:
import pandas as pd
from util import ANNOTATED_FILE_JAVA_COMPLET

# Ler os dois arquivos .xlsx
df_ano = pd.read_excel(ANNOTATED_FILE_JAVA_COMPLET, keep_default_na=False, sheet_name='2021')
df1 = df_ano[df_ano['discardReason'] == ''] #filtra os projetos sem discardReason
#df1 = pd.read_excel('arquivo2.xlsx')  # DataFrame 2

# Verificar quais itens da coluna 'name' em df1 estão presentes na coluna 'projects' de df2
itens_presentes = df2['name'].isin(df1['name'])

# Exibir o resultado: True para os itens que estão presentes, False para os que estão faltando
#print(itens_presentes)

# Filtrar os itens que estão presentes
nomes_presentes = df2[itens_presentes]
#print("Itens presentes em ambas as colunas:")
#print(nomes_presentes)

# Filtrar os itens que estão faltando
nomes_faltando = df2[~itens_presentes]
#print("\nItens da coluna 'name' que estão faltando em 'projects':")
#print(nomes_faltando)

# Converter as colunas para sets
names_set = set(df1['name'])
projects_set = set(df2['name'])

# Verificar se os conjuntos são iguais
if names_set == projects_set:
    print("Os itens nas colunas 'name' e 'projects' são os mesmos (em termos de valores).")
else:
    print("As colunas 'name' e 'projects' têm valores diferentes.")
    
# Itens presentes em 'name' mas não em 'projects'
names_faltando = names_set - projects_set
print("\nItens presentes em 'name' mas não em 'projects':", names_faltando)

# Itens presentes em 'projects' mas não em 'name'
projects_faltando = projects_set - names_set
print("Itens presentes em 'projects' mas não em 'name':", projects_faltando)

if len(df1['name']) == len(df2['name']):
    print("\nAs quantidades de itens nas colunas 'name' e 'projects' são iguais.")
else:
    print("\nAs quantidades de itens nas colunas 'name' e 'projects' são diferentes.")

duplicados_projects = df2['name'].duplicated(keep=False)

# Filtrar os projetos duplicados
projetos_duplicados = df2.loc[duplicados_projects, 'name'].unique()

if len(projetos_duplicados) > 0:
    print(f"\nProjetos duplicados: {', '.join(projetos_duplicados)}")
else:
    print("\nNão existem itens duplicados na coluna 'projects'.")

Os itens nas colunas 'name' e 'projects' são os mesmos (em termos de valores).

Itens presentes em 'name' mas não em 'projects': set()
Itens presentes em 'projects' mas não em 'name': set()

As quantidades de itens nas colunas 'name' e 'projects' são iguais.

Projetos duplicados: android


New dataframe filtering only lines that first version projects

In [7]:
#df10 = df2.mask(df2 > 1, 1)
df10 = df[(df['part_commit'] == 1)]
df10

,owner,name,domain,sha1,part_commit,date_commit,isLast,SapHana,Realm,SAP SQL Anywhere,...,Firebird,Cassandra,Couchbase,Etcd,PostGIS,Redis,Join_Ignite-Sql,Join_Ignite-NoSql,Join_Virtuoso-Sql,Join_Virtuoso-NoSql
0,Activiti,Activiti,Enterprise Resource Planning,7262dc3913045001cbc658a8474e9d295980c345,1,2010-07-14 06:36:18+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,0,0
55,airbnb,lottie-android,Game,13073ebe409bc09aa71afec25f391fd3a340588a,1,2016-09-24 19:32:20+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,0,0
71,alibaba,spring-cloud-alibaba,Software Development,bfae54b51e47a616fa76ce5a01d787949ac44eb3,1,2018-11-20 09:17:44+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,0,0
81,alibaba,nacos,Infrastructure Management,3ab06bc1dd75c53e3efe968c83384ded4330d4c2,1,2018-12-07 06:30:11+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,0,0
104,alibaba,fastjson,Data Management,501e27c4aa8796607ddca64938a85626fd43610b,1,2011-09-14 13:58:01+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22716,yacy,yacy_search_server,Infrastructure Management,cfe3d7e640f1aa86832b1da28c4b3908121499e5,1,2005-05-10 10:11:51+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,0,0
22817,zaproxy,zaproxy,Security,bd4f7c5d2b2ebac1eb1ce70cd733d09b8165d728,1,2011-01-06 18:32:48+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,0,0
22853,zendesk,maxwell,Data Management,6339da9d6d6dbc69f3973466a8c7542d214322ef,1,2015-03-09 23:10:54+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,0,0
22867,zxing,zxing,Software Development,1ec82dd376409fa723a9b8eb7afc8011797631c5,1,2008-01-22 18:59:47+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Removing projects that don't have DB clues the principal dataframe 

In [8]:
#df2 = df2.iloc[ : , 1:]
#df2[(df2 == 0).all(axis=1)].index
df2 = df2.drop(df2[(df2.iloc[ : , 8:] == 0).all(axis=1)].index)
df2

,owner,name,part_commit,isLast,SapHana,Realm,SAP SQL Anywhere,FileMaker,ArangoDB,Informix,...,Firebird,Cassandra,Couchbase,Etcd,PostGIS,Redis,Join_Ignite-Sql,Join_Ignite-NoSql,Join_Virtuoso-Sql,Join_Virtuoso-NoSql
0,Activiti,Activiti,1540,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Alluxio,alluxio,5886,0,0,0,0,0,0,0,...,0,0,0,4,0,0,0,0,0,0
4,AnySoftKeyboard,AnySoftKeyboard,703,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,Athou,commafeed,465,0,0,0,0,0,0,0,...,0,0,0,0,0,18,0,0,0,0
7,Atmosphere,atmosphere,1326,0,0,0,0,0,0,0,...,0,0,0,0,0,26,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
355,wildfly,wildfly,27495,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
357,xuxueli,xxl-job,120,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
358,yacy,yacy_search_server,5151,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
359,zaproxy,zaproxy,666,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Removing projects that don't have DB clues in the first version 

In [9]:
df3 = df[(df['part_commit'] == 1)]
#df3.drop(["PROJECTS", "COMMITS"], axis=1, inplace=True)
df3 = df3.drop(df3[(df3.iloc[:,8:] == 0).all(axis=1)].index) # deletando as linhas que possuem valor 0 em todas as colunas
df3

,owner,name,domain,sha1,part_commit,date_commit,isLast,SapHana,Realm,SAP SQL Anywhere,...,Firebird,Cassandra,Couchbase,Etcd,PostGIS,Redis,Join_Ignite-Sql,Join_Ignite-NoSql,Join_Virtuoso-Sql,Join_Virtuoso-NoSql
0,Activiti,Activiti,Enterprise Resource Planning,7262dc3913045001cbc658a8474e9d295980c345,1,2010-07-14 06:36:18+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,0,0
81,alibaba,nacos,Infrastructure Management,3ab06bc1dd75c53e3efe968c83384ded4330d4c2,1,2018-12-07 06:30:11+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,0,0
104,alibaba,fastjson,Data Management,501e27c4aa8796607ddca64938a85626fd43610b,1,2011-09-14 13:58:01+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,0,0
428,apache,rocketmq,Data Management,b29c318cdde225ef3a33a73e939e49e087766a28,1,2017-01-21 15:13:01+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,0,0
454,apache,hudi,Data Management,77e54e78f8cf4f615f1b899703c85b00a8f8aad1,1,2017-03-27 21:49:26+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22052,VertaAI,modeldb,Artificial Intelligence,103c16938d2c49a57620c09a6de9a0d0ed405635,1,2016-11-03 18:44:07+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,0,0
22701,xuxueli,xxl-job,Automation,66adbd687faf8ae1298d9f2268dec2aee65cec95,1,2016-05-20 13:05:25+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,0,0
22817,zaproxy,zaproxy,Security,bd4f7c5d2b2ebac1eb1ce70cd733d09b8165d728,1,2011-01-06 18:32:48+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,0,0
22853,zendesk,maxwell,Data Management,6339da9d6d6dbc69f3973466a8c7542d214322ef,1,2015-03-09 23:10:54+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Merging dataframe grouping and dataframe only first version with DB clues

In [10]:
#df3.merge(df4, on=['PROJECTS'], how='outer', suffixes=['', '_'], indicator=True)
df_mer= pd.merge(df2, df3, on= 'name', how='outer', suffixes=['', '_'], indicator=True)
df_mer

,owner,name,part_commit,isLast,SapHana,Realm,SAP SQL Anywhere,FileMaker,ArangoDB,Informix,...,Cassandra_,Couchbase_,Etcd_,PostGIS_,Redis_,Join_Ignite-Sql_,Join_Ignite-NoSql_,Join_Virtuoso-Sql_,Join_Virtuoso-NoSql_,_merge
0,Activiti,Activiti,1540,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,both
1,Alluxio,alluxio,5886,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
2,AnySoftKeyboard,AnySoftKeyboard,703,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
3,Athou,commafeed,465,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,both
4,Atmosphere,atmosphere,1326,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
225,wildfly,wildfly,27495,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
226,xuxueli,xxl-job,120,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,both
227,yacy,yacy_search_server,5151,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
228,zaproxy,zaproxy,666,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,both


Filtering diference in the first version

In [11]:
df5 = df_mer[(df_mer['_merge'].str.startswith('left'))]
df5

,owner,name,part_commit,isLast,SapHana,Realm,SAP SQL Anywhere,FileMaker,ArangoDB,Informix,...,Cassandra_,Couchbase_,Etcd_,PostGIS_,Redis_,Join_Ignite-Sql_,Join_Ignite-NoSql_,Join_Virtuoso-Sql_,Join_Virtuoso-NoSql_,_merge
1,Alluxio,alluxio,5886,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
2,AnySoftKeyboard,AnySoftKeyboard,703,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
4,Atmosphere,atmosphere,1326,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
7,FasterXML,jackson-databind,1596,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
8,GoogleContainerTools,jib,276,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
222,vipshop,Saturn,153,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
223,vsch,flexmark-java,136,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
224,webbukkit,dynmap,253,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
225,wildfly,wildfly,27495,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only


Merging dataframe only lines that first version projects and dataframe only lines in the first version and not DB clues

In [12]:
df_mer2= pd.merge(df10, df5.iloc[:,:2], on= 'name', how='outer', suffixes=['', '_'], indicator=True)
df_mer2

,owner,name,domain,sha1,part_commit,date_commit,isLast,SapHana,Realm,SAP SQL Anywhere,...,Couchbase,Etcd,PostGIS,Redis,Join_Ignite-Sql,Join_Ignite-NoSql,Join_Virtuoso-Sql,Join_Virtuoso-NoSql,owner_,_merge
0,Activiti,Activiti,Enterprise Resource Planning,7262dc3913045001cbc658a8474e9d295980c345,1,2010-07-14 06:36:18+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,NaN,left_only
1,airbnb,lottie-android,Game,13073ebe409bc09aa71afec25f391fd3a340588a,1,2016-09-24 19:32:20+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,NaN,left_only
2,alibaba,spring-cloud-alibaba,Software Development,bfae54b51e47a616fa76ce5a01d787949ac44eb3,1,2018-11-20 09:17:44+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,alibaba,both
3,alibaba,nacos,Infrastructure Management,3ab06bc1dd75c53e3efe968c83384ded4330d4c2,1,2018-12-07 06:30:11+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,NaN,left_only
4,alibaba,fastjson,Data Management,501e27c4aa8796607ddca64938a85626fd43610b,1,2011-09-14 13:58:01+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,NaN,left_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
359,yacy,yacy_search_server,Infrastructure Management,cfe3d7e640f1aa86832b1da28c4b3908121499e5,1,2005-05-10 10:11:51+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,yacy,both
360,zaproxy,zaproxy,Security,bd4f7c5d2b2ebac1eb1ce70cd733d09b8165d728,1,2011-01-06 18:32:48+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,NaN,left_only
361,zendesk,maxwell,Data Management,6339da9d6d6dbc69f3973466a8c7542d214322ef,1,2015-03-09 23:10:54+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,NaN,left_only
362,zxing,zxing,Software Development,1ec82dd376409fa723a9b8eb7afc8011797631c5,1,2008-01-22 18:59:47+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,NaN,left_only


Filtering only lines in the first version and not DB clues

In [13]:
df6 = df_mer2[(df_mer2['_merge'].str.startswith('bo'))]
df6

,owner,name,domain,sha1,part_commit,date_commit,isLast,SapHana,Realm,SAP SQL Anywhere,...,Couchbase,Etcd,PostGIS,Redis,Join_Ignite-Sql,Join_Ignite-NoSql,Join_Virtuoso-Sql,Join_Virtuoso-NoSql,owner_,_merge
2,alibaba,spring-cloud-alibaba,Software Development,bfae54b51e47a616fa76ce5a01d787949ac44eb3,1,2018-11-20 09:17:44+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,alibaba,both
5,aliyun,aliyun-openapi-java-sdk,Infrastructure Management,7685188c98e5271e3e32fcdfcf3f0b01db8b98fa,1,2016-08-18 15:42:32+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,aliyun,both
6,Alluxio,alluxio,Data Management,3050002ebc4defeb60ee350cae24281bf735ad29,1,2013-03-13 05:52:38+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,Alluxio,both
10,AnySoftKeyboard,AnySoftKeyboard,Other,4c8780f1988a92ac0ac9953c5cc331f988bf4c97,1,2009-09-03 17:59:09+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,AnySoftKeyboard,both
13,apache,iceberg,Data Management,abab0bbdf566b2472cf87b864687900913d1a456,1,2018-04-11 19:36:50+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,apache,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
353,vipshop,Saturn,Automation,079a93ed3aa4c2cae211c623b23db1b39ca1291b,1,2016-12-21 06:13:44+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,vipshop,both
354,vsch,flexmark-java,Software Development,cda4ca57806fd1350c2a172e1af2c442448e9f7b,1,2015-06-14 13:55:01+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,vsch,both
355,webbukkit,dynmap,Game,455b5d3b3e9a71239eb4fcb962c72876425ee9ed,1,2011-02-01 14:16:54+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,webbukkit,both
356,wildfly,wildfly,Infrastructure Management,ed53cd2e0e96db369c3b26b65cf397ace08b86b4,1,2010-08-13 14:18:14+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,wildfly,both


Concatening dataframes

In [14]:
df_concat2 =  pd.concat([df3, df6]).drop_duplicates()
df_concat2

,owner,name,domain,sha1,part_commit,date_commit,isLast,SapHana,Realm,SAP SQL Anywhere,...,Couchbase,Etcd,PostGIS,Redis,Join_Ignite-Sql,Join_Ignite-NoSql,Join_Virtuoso-Sql,Join_Virtuoso-NoSql,owner_,_merge
0,Activiti,Activiti,Enterprise Resource Planning,7262dc3913045001cbc658a8474e9d295980c345,1,2010-07-14 06:36:18+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,NaN,NaN
81,alibaba,nacos,Infrastructure Management,3ab06bc1dd75c53e3efe968c83384ded4330d4c2,1,2018-12-07 06:30:11+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,NaN,NaN
104,alibaba,fastjson,Data Management,501e27c4aa8796607ddca64938a85626fd43610b,1,2011-09-14 13:58:01+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,NaN,NaN
428,apache,rocketmq,Data Management,b29c318cdde225ef3a33a73e939e49e087766a28,1,2017-01-21 15:13:01+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,NaN,NaN
454,apache,hudi,Data Management,77e54e78f8cf4f615f1b899703c85b00a8f8aad1,1,2017-03-27 21:49:26+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
353,vipshop,Saturn,Automation,079a93ed3aa4c2cae211c623b23db1b39ca1291b,1,2016-12-21 06:13:44+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,vipshop,both
354,vsch,flexmark-java,Software Development,cda4ca57806fd1350c2a172e1af2c442448e9f7b,1,2015-06-14 13:55:01+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,vsch,both
355,webbukkit,dynmap,Game,455b5d3b3e9a71239eb4fcb962c72876425ee9ed,1,2011-02-01 14:16:54+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,webbukkit,both
356,wildfly,wildfly,Infrastructure Management,ed53cd2e0e96db369c3b26b65cf397ace08b86b4,1,2010-08-13 14:18:14+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,wildfly,both


In [15]:
#df2 = df2.astype(np.uint8)

In [16]:
df_concat2.drop(["owner","name", "domain", "sha1", "part_commit", "date_commit", "isLast", "owner_", "_merge"], axis=1, inplace=True)
df_concat2

,SapHana,Realm,SAP SQL Anywhere,FileMaker,ArangoDB,Informix,HyperSQL,CouchDB,Riak KV,Neo4j,...,Firebird,Cassandra,Couchbase,Etcd,PostGIS,Redis,Join_Ignite-Sql,Join_Ignite-NoSql,Join_Virtuoso-Sql,Join_Virtuoso-NoSql
0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
81,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
104,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
428,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
454,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
353,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
354,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
355,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
356,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


ASSOCIATION RULES

In [17]:
frq_items = apriori(df_concat2, min_support = 0.021, use_colnames = True)

In [18]:
rules = association_rules(frq_items, metric ="lift", min_threshold = 1)

In [19]:
# Sorting the rules by the lift measure 
rules = rules.sort_values('lift', ascending =False)
print(rules.head())

                                   antecedents                consequents  \
2308         (Oracle, HyperSQL, MS SQL Server)       (H2, MySQL, IBM DB2)   
2618         (Oracle, HyperSQL, MS SQL Server)  (H2, PostgreSQL, IBM DB2)   
1634     (HyperSQL, MS SQL Server, PostgreSQL)              (H2, IBM DB2)   
2751     (HyperSQL, MS SQL Server, PostgreSQL)       (H2, MySQL, IBM DB2)   
2295  (Oracle, HyperSQL, MS SQL Server, MySQL)              (H2, IBM DB2)   

      antecedent support  consequent support   support  confidence       lift  \
2308            0.025862            0.021552  0.021552    0.833333  38.666667   
2618            0.025862            0.021552  0.021552    0.833333  38.666667   
1634            0.025862            0.021552  0.021552    0.833333  38.666667   
2751            0.025862            0.021552  0.021552    0.833333  38.666667   
2295            0.025862            0.021552  0.021552    0.833333  38.666667   

      leverage  conviction  
2308  0.020994     5.

In [20]:
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
2308,"(Oracle, HyperSQL, MS SQL Server)","(H2, MySQL, IBM DB2)",0.025862,0.021552,0.021552,0.833333,38.666667,0.020994,5.870690
2618,"(Oracle, HyperSQL, MS SQL Server)","(H2, PostgreSQL, IBM DB2)",0.025862,0.021552,0.021552,0.833333,38.666667,0.020994,5.870690
1634,"(HyperSQL, MS SQL Server, PostgreSQL)","(H2, IBM DB2)",0.025862,0.021552,0.021552,0.833333,38.666667,0.020994,5.870690
2751,"(HyperSQL, MS SQL Server, PostgreSQL)","(H2, MySQL, IBM DB2)",0.025862,0.021552,0.021552,0.833333,38.666667,0.020994,5.870690
2295,"(Oracle, HyperSQL, MS SQL Server, MySQL)","(H2, IBM DB2)",0.025862,0.021552,0.021552,0.833333,38.666667,0.020994,5.870690
...,...,...,...,...,...,...,...,...,...
22,(H2),(MySQL),0.116379,0.232759,0.073276,0.629630,2.705075,0.046188,2.071552
9,(MySQL),(HyperSQL),0.232759,0.137931,0.086207,0.370370,2.685185,0.054102,1.369168
8,(HyperSQL),(MySQL),0.137931,0.232759,0.086207,0.625000,2.685185,0.054102,2.045977
51,(MySQL),(SQLite),0.232759,0.051724,0.030172,0.129630,2.506173,0.018133,1.089508


In [21]:
rules['antecedent_len'] = rules['antecedents'].apply(lambda x: len(x))
rules['consequente_len'] = rules['consequents'].apply(lambda x: len(x))

Filtering only the rules of size 2, that is, 1 antecedent and 1 consequent. 

In [22]:
rules_filter = rules[(rules['antecedent_len'] == 1) & (rules['consequente_len'] == 1)]

Showing all filtered rules.

In [23]:
rules_filter

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len,consequente_len
57,(IBM DB2),(MS SQL Server),0.043103,0.038793,0.021552,0.500000,12.888889,0.019880,1.922414,1,1
56,(MS SQL Server),(IBM DB2),0.038793,0.043103,0.021552,0.555556,12.888889,0.019880,2.153017,1,1
60,(MS SQL Server),(SQLite),0.038793,0.051724,0.021552,0.555556,10.740741,0.019545,2.133621,1,1
61,(SQLite),(MS SQL Server),0.051724,0.038793,0.021552,0.416667,10.740741,0.019545,1.647783,1,1
37,(IBM DB2),(Oracle),0.043103,0.086207,0.038793,0.900000,10.440000,0.035077,9.137931,1,1
...,...,...,...,...,...,...,...,...,...,...,...
22,(H2),(MySQL),0.116379,0.232759,0.073276,0.629630,2.705075,0.046188,2.071552,1,1
9,(MySQL),(HyperSQL),0.232759,0.137931,0.086207,0.370370,2.685185,0.054102,1.369168,1,1
8,(HyperSQL),(MySQL),0.137931,0.232759,0.086207,0.625000,2.685185,0.054102,2.045977,1,1
51,(MySQL),(SQLite),0.232759,0.051724,0.030172,0.129630,2.506173,0.018133,1.089508,1,1


In [24]:
HISTORICAL_RULES_FILE = RESOURCE_DIR + os.sep + 'historical_rulesv1.xlsx'  #'seq_patterns/testes/historical_rules_commit1_197reg_021.xlsx'
rules_filter = rules_filter.to_excel(HISTORICAL_RULES_FILE, index=False)